# TME 3.2: Projet Detection de motifs


## Recheche de pattern (motifs) en utilisant les algoritmes randomisés

Les algorithmes randomisés prendre des décisions aléatoire plutôt que déterministes.
l'algorithme s'exécute différemment à chaque fois. Ils sont couramment utilisés dans situations où aucun algorithme exact et rapide est connu. Nous allons d'abord implémenter l'algorithm random Projections.


1\.  Nous allons réutiliser les fonctions du precedent pour générer t séquences artificielles de taille n, et implanter dans chaque séquence un motif de taille k à des positions aléatoires avec v substitutions choisies aléatoirement. Nous allons faire varier le motifs dans 50% de cas.

In [1]:
import random

nuc = ('A', 'C', 'G', 'T')

k=7 #taille de motif
v=1 #nb de positions variable dans le motif
t=10 #nb de sequences
n=10 #longuer des sequence
f=0.5

In [2]:
def insertMotif(sequence, motif, position):
    return sequence[:position] + motif + sequence[position:]

def generateRandomSequence(n:int, upper=False):
    """
    Même chose hein
    """
    sequence = "".join([random.choice(nuc) for _ in range(n)])
    if upper:
        return sequence
    return sequence.lower()

def modifierMotif(motif:str, nbpos:int,  upper=False):
    """
    Modifie nbpos positions d'un motif aléatoirement 
    entrée motif: motif à modifier
    entrée nbpos: nombre de positions
    entrée upper : bool, si True les nucléotides modifiés seront minuscule, False majuscule
    sortie motifM: motif modifié
    """
    motifM = list(motif)

    nbPos_real = min(nbpos, len(motif))
    allPos = list(range(len(motif)))

    for _ in range(nbPos_real):
        index = random.choice(range(len(allPos)))
        changeIndex = allPos[index]
        del allPos[index]

        nv_L = generateRandomSequence(1, upper)
        motifM[changeIndex] = nv_L
    return "".join(motifM)        


def implantMotifVar(k, v, t, n, f):
    """
    Génère des séquences aléatoires et les implante des motifs variables (un motif par séquence)
    entrée k: taille du motif
    entrée v: nombre de variations
    entrée t : nombre de séquences 
    entrée n : longueur des séquences
    entrée f : frequence de variation du motif.
    sortie DNA : matrice de dimension txn avec les motifs implantés
    REMARQUE : La taille totale des séquences plus motif doit être égal à t, pensez à générer de séquence aléatoire de taille t-k pour pouvoir implanter un motif de taille k
    """

    sequences = []

    base_motif = generateRandomSequence(k)
    print('motif : ' + str(base_motif))
    for _ in range(t):
        base_seq = generateRandomSequence(n - k)

        changed_motif = modifierMotif(base_motif, v) if random.random() < f else base_motif
        
        sequences.append(insertMotif(base_seq, changed_motif, random.choice(range(len(base_seq)))))
    
    return sequences

adn = implantMotifVar(k, v, t, n, f)
print (adn)

adn  = [s.upper() for s in adn]
print (adn)

motif : atctcat
['tatctcattg', 'tatcccattg', 'tgatctcatg', 'aaatctcatc', 'atcttatgcc', 'aatctcataa', 'catcttatat', 'catctcatac', 'aatcgcatct', 'atctcattga']
['TATCTCATTG', 'TATCCCATTG', 'TGATCTCATG', 'AAATCTCATC', 'ATCTTATGCC', 'AATCTCATAA', 'CATCTTATAT', 'CATCTCATAC', 'AATCGCATCT', 'ATCTCATTGA']


2\. Nous allons implémenter l'algorithme ``randomProjection``. D'abord, faites la fonction `getRandomFixePositions` pour générer une projection de p à k, voir un exemple dans les slides de cours. Faire aussi la fonction `generateKey` qui extrait les caractères du motif puis génère une cle qui représente la projection.

In [3]:
def getRandomFixePositions(k, p):
    """
    Genere une projection de p vers k
    entrée p: nombre de positions choisi pour la projection 
    entrée k: nombre de positions du motif original
    sortie projection: liste de positions choisi aléatoirement
    """
    positions = set(range(k))
    projection = list()

    for _ in range(p):
        projection.append(random.choice(list(positions)))
        positions.remove(projection[-1])
   
    projection.sort()
    return projection

lR = getRandomFixePositions(7, 4)
print(lR)

def generateKey(projection, motif):
    """
    extrait les caractères du motif et génère la cle de la projection
    entrée projection : liste de positions qui represent la projection
    entrée motif : motif de taille k
    sortie cle : cle de la projection
    """
    return "".join([motif[i] for i in projection])

generateKey(lR, "01234567")

[0, 1, 4, 6]


'0146'

3\. Implémenter l'algorithme ``randomProjection``. Bonus : Pour ameliorer la performance vous pouvez abandonner les motifs de taille k peu complexes.

In [4]:
from itertools import product

def removeLowComplexe(motifs, size):
    """
    Eleve les motifs peu complexe ayant 
    entrée motifs: dictionnaire de motifs, clé=motif, valeur = fréquence d'observation
    sortie motifsClean: dictionnaire de motifs sans les motifs peu complexe.
    """
    motifsClean = []
    twoLetterCombs = ["".join(tup) for tup in list(product(["A", "T", "G", "C"], repeat=2))]
    for motif in [m.upper() for m in motifs]:
        if motif.count("A") > size or motif.count("T") > size or motif.count("G") > size or motif.count("C") > size:
            continue
        if True in [x*3 in motif for x in twoLetterCombs]:
            print(motif)
            continue
        motifsClean.append(motif)
        
    return motifsClean

In [44]:
def randomProjection(k, p, sequences):
    """
    Implémente l'algorithme randomProjection
    entrée k : taille du motif
    entrée p : nombre de positions de la projection 
    entrée sequences : matrice de dimension txn contenant les séquences 
    sortie motifs : dictionaire, cle = projection, valeur= frequence
    sortie motifsSeq:  dictionaire, cle = projection, valeur = LISTE DES MOTIFS ORIGINAUX (pas 'valeur = original motif' ca n'a pas de sens)
    """
    motifs = {}; motifsSeq = {}
    projection = getRandomFixePositions(k, p)

    for seq in sequences:
        for pos in range(len(seq)-k+1):
            currentMotif = seq[pos:pos+k]
            key = generateKey(projection, currentMotif)
            
            if(key in motifs and currentMotif not in motifsSeq[key]):
                motifs[key] += 1
                motifsSeq[key].append(currentMotif)
            else:
                motifs[key] = 1
                motifsSeq[key] = [currentMotif]
    
    return motifs, motifsSeq

#motifsSort = sorted(motifs, reverse=True, key=motifs.get)

adnTest = ['TTAACGGCAC', 'GCTCACGCCA', 'TACCGGCCGT']
motifsProj, motifsSeq = randomProjection(7, 4, adnTest)
print (motifsProj)
print (motifsSeq)

#motifsProj => {'TACG': 1, 'TCGC': 3, 'AGGA': 1, 'AGCC': 1, 'GCAG': 1, 'CACC': 1, 'CGCA': 1, 'AGGC': 1, 'CGCG': 1, 'CCCT': 1}
#motifsSeq => {'TACG': ['TTAACGG'], 'TCGC': ['TAACGGC', 'TCACGCC', 'TACCGGC'], 'AGGA': ['AACGGCA'], 'AGCC': ['ACGGCAC'], 'GCAG': ['GCTCACG'], 'CACC': ['CTCACGC'], 'CGCA': ['CACGCCA'], 'AGGC': ['ACCGGCC'], 'CGCG': ['CCGGCCG'], 'CCCT': ['CGGCCGT']}

{'TAAG': 1, 'AACC': 1, 'ACGA': 2, 'CGGC': 1, 'CTCG': 1, 'TCAC': 1, 'CACC': 1, 'ACCC': 1, 'CCGC': 1, 'CGGG': 1, 'GGCT': 1}
{'TAAG': ['TTAACGG'], 'AACC': ['TAACGGC'], 'ACGA': ['AACGGCA', 'CACGCCA'], 'CGGC': ['ACGGCAC'], 'CTCG': ['GCTCACG'], 'TCAC': ['CTCACGC'], 'CACC': ['TCACGCC'], 'ACCC': ['TACCGGC'], 'CCGC': ['ACCGGCC'], 'CGGG': ['CCGGCCG'], 'GGCT': ['CGGCCGT']}


In [45]:
motifsProj, motifsSeq = randomProjection(k, 4, adn)
print(motifsProj)
print(motifsSeq)

{'ATTC': 2, 'TCCA': 1, 'CTAT': 2, 'TCTT': 1, 'ATCC': 1, 'CCAT': 1, 'CCTT': 1, 'GACT': 1, 'AACT': 1, 'TCTA': 2, 'TTTG': 1, 'TAGC': 1, 'ATTT': 1, 'TTTA': 1, 'ATGC': 1, 'CGAT': 1, 'GCTC': 1, 'CATG': 1}
{'ATTC': ['AATCTCA', 'CATCTCA'], 'TCCA': ['ATCTCAT'], 'CTAT': ['TCTCATA', 'TCTCATT'], 'TCTT': ['CTCATTG'], 'ATCC': ['TATCCCA'], 'CCAT': ['TCCCATT'], 'CCTT': ['CCCATTG'], 'GACT': ['TGATCTC'], 'AACT': ['AAATCTC'], 'TCTA': ['ATCTTAT', 'CTCATAC'], 'TTTG': ['CTTATGC'], 'TAGC': ['TTATGCC'], 'ATTT': ['CATCTTA'], 'TTTA': ['CTTATAT'], 'ATGC': ['AATCGCA'], 'CGAT': ['TCGCATC'], 'GCTC': ['CGCATCT'], 'CATG': ['TCATTGA']}


4\. Avez vous trouvez le motif implanté? Rexécuter l’algorithme plusieurs fois pour augmenter les chances de le trouver. 

**Reponse :** Oui, la fonction trouve bien les séquences implantées. 
Dans cet example, on a généré 10 séquences avec le motif implanté **actccat**:
['tgactccatg', 'taactccaat', 'cactccatct', 'actccatgat', 'cactccattg', 'ccactccatg', 'cactccatcc', 'aacctccatt', 'aactccatta', 'aactccatta']

Voici les dictionnaires **motifsProj** et **motifsSeq** générés après application de l'algorithme RandomProjection sur ces 10 séquences:
{'TGAT': 1, 'GACC': 1, **'ACTC': 9, 'CTCA': 10**, 'TAAT': 1, 'AACC': 3, 'CACC': 4, 'TCCT': 6, 'CCAG': 1, 'CCAT': 1, 'AACT': 1, 'ACCC': 1, 'CCTC': 1}
{
    'TGAT': ['TGACTCC'], 
    'GACC': ['GACTCCA'], 
    **'ACTC': ['ACTCCAT', 'ACTCCAA', 'ACTCCAT', 'ACTCCAT', 'ACTCCAT', 'ACTCCAT', 'ACTCCAT', 'ACTCCAT', 'ACTCCAT'],** 
    **'CTCA': ['CTCCATG', 'CTCCAAT', 'CTCCATC', 'CTCCATG', 'CTCCATT', 'CTCCATG', 'CTCCATC', 'CTCCATT', 'CTCCATT', 'CTCCATT']**, 
    'TAAT': ['TAACTCC'], 
    'AACC': ['AACTCCA', 'AACTCCA', 'AACTCCA'], 
    'CACC': ['CACTCCA', 'CACTCCA', 'CACTCCA', 'CACTCCA'],
    'TCCT': ['TCCATCT', 'TCCATGA', 'TCCATTG', 'TCCATCC', 'TCCATTA', 'TCCATTA'], 
    'CCAG': ['CCATGAT'], 
    'CCAT': ['CCACTCC'], 
    'AACT': ['AACCTCC'], 
    'ACCC': ['ACCTCCA'], 
    'CCTC': ['CCTCCAT']
}


5\. Implémenter la version itérative de l’algorithme ``randomProjection``. 

In [46]:
#Construire matrice de fréquence
def profile(motifs, k, nuc):
    """
    Construire une matrice de fréquence de dimension k x |nuc|
    entrée motifs: liste de motifs
    entrée k: taille du motif
    entrée nuc: alphabet
    sortie MF: matrice de fréquence
    """
    matrice = list()
    for i, nucleotide in enumerate(nuc):
        matrice.append(list())
        for j in range(k):
            matrice[i].append(len([1 for motif in motifs if motif[j] == nucleotide]))
    
    return matrice

def getScore(MF, k):
    """
    Renvoie le score de MF, la somme des max de chaque colonne
    entrée MF: matrice de fréquence
    entrée k: taille du motif
    sortie sc: score
    """
    return sum([max([line[i] for line in MF]) for i in range(k)])

In [52]:
def randomProjIt(sequences, k, v, nuc, It):
    """
    Implémente l'algorithme randomProjection version iterative
    entrée sequences : matrice de dimension txn contenant les séquences 
    entrée k : nombre de positions du motif
    entrée v : nombre de positions de la projection 
    entrée nuc : alphabet
    entrée It: nombre d'iterations
    sortie score : meilleur score
    sortie motifs :  liste de motifs associés au meilleur score
    """
 
    motifs = []; score = 0

    for _ in range(It):
        motifsProj, motifsSeq = randomProjection(k, v, sequences)
        for bucket in motifsSeq.keys():
            motif_list = motifsSeq[bucket]
            matrice = profile(motif_list, k, nuc)
            current_score = getScore(matrice, k)
            if current_score > score:
                motifs = motif_list
                score = current_score
    
    return score, motifs

score, seqsMotif = randomProjIt(adn, 7, 4, nuc, 100)

print (score, seqsMotif)


24 ['AATCTCA', 'CATCTTA', 'CATCTCA', 'AATCGCA']


6\. Tester l'algorithme  ``randomProjection`` sur vos données de chipSeq. N'oubliez pas de chercher les motifs dans le brin complémentaire et faire un merge de résultats. Puis générér le LOGO du motif trouvé.

In [48]:
def reverseComplement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in reversed(seq)])

def complement(seq):
    seq_dict = {'A':'T','T':'A','G':'C','C':'G'}
    return "".join([seq_dict[base] for base in seq])

def printTopFMotifsFreq(motifs, m, rev=False):
    motifsRet = {}
    motifsSort = sorted(motifs, reverse=True, key=motifs.get)
    i = 0
    while (i < m):
        motifPrint = motifsSort[i]
        print (motifsSort[i])
        if rev:
            motifPrint = reverseComplement(motifsSort[i])
        print (i, motifPrint, "-", motifs[motifsSort[i]])
        i = i + 1

In [55]:
k=8; p=4; n=80

def readFasta(genome, n):
    sequence = []
    file = open(genome, "r")
    sequence = []
    for s in file:
        if s[0] != ">":
            sequence.append(s.strip().upper())
    sequenceStr = "".join(sequence)
    #sequence = [sequenceStr]
    sequence = [sequenceStr[i:i+n] for i in range(0, len(sequenceStr), n)]
    sequenceRet = [x for x in sequence if x]
    return sequenceRet

genome = "Sequence_by_Peaks_3.fasta"

sequencesChip   = readFasta(genome, n)
t = len(sequencesChip)
print (sequencesChip[8], t, n, k)
revSequences = [reverseComplement(m) for m in sequencesChip]

sequences = sequencesChip + revSequences

score, seqsMotif = randomProjIt(sequences, k, p, nuc, 100)

AAGTTGTAGTACTTGTGTTCCTGGTTTGCTGGTTTGCTTGGTTTGTCGGTGTATCTGCAGTTGTGTATTTGCAGGTCTGT 577 80 8


In [57]:
score, seqsMotif, len(seqsMotif)

(308,
 ['TTTGACCT',
  'TTTGAACT',
  'GTTGGGCT',
  'TTTGGGCT',
  'TTTGGTCT',
  'ATTATACT',
  'ATTGAGCT',
  'ATTAATCT',
  'GTTAAACT',
  'CTTTTCCT',
  'GTTTCTCT',
  'CTTAACCT',
  'ATTGCCCT',
  'TTTATCCT',
  'GTTCAACT',
  'TTTTCTCT',
  'ATTTGTCT',
  'TTTTGGCT',
  'ATTTGCCT',
  'GTTGGTCT',
  'GTTTTCCT',
  'TTTGCTCT',
  'CTTCCTCT',
  'ATTTCCCT',
  'ATTGTGCT',
  'CTTTTGCT',
  'TTTCATCT',
  'CTTATGCT',
  'ATTATCCT',
  'GTTAGACT',
  'CTTGCTCT',
  'CTTCATCT',
  'ATTCCTCT',
  'TTTTTTCT',
  'GTTAGTCT',
  'TTTAATCT',
  'GTTTCACT',
  'GTTGAACT',
  'GTTGAGCT',
  'CTTGCCCT',
  'CTTTCTCT',
  'TTTTCCCT',
  'ATTTTTCT',
  'CTTATCCT',
  'CTTTTTCT',
  'TTTATGCT',
  'CTTTATCT',
  'GTTCTGCT',
  'TTTACACT',
  'ATTGGACT',
  'GTTACACT',
  'ATTGTCCT',
  'CTTCTTCT',
  'GTTCTACT',
  'GTTTTTCT',
  'TTTCTGCT',
  'TTTATACT',
  'CTTTGTCT'],
 58)